In [1]:
import numpy as np
import pandas as pd
import string
import unicodedata
import re
import spacy

# Uncomment to see full cell text
pd.set_option('display.max_colwidth', 0)

# Preprocess Raw Text Data
Use spaCy to tokenize sentences

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [65]:
raw = pd.read_excel("../data/raw/inquire-boulder-faq-text/inquire-boulder-active-faqs-2019-01-02.xls.xlsx")

In [7]:
def preprocess(entry):
    entry = entry.replace('\n', ' ')
    entry = unicodedata.normalize("NFKD", entry)
    entry = entry.replace('?', '? ')
    return entry + ' '

def sentence_tokenize(entry):
    doc = nlp(entry)
    sentences = list(doc.sents)
    sentences = '\n\n'.join([(s.text) for s in sentences])
    return sentences

In [8]:
faq = raw.drop(labels=('Active'), axis=1).dropna()
faq.FAQ = faq.FAQ.apply(preprocess)
faq.FAQ = faq.FAQ.apply(sentence_tokenize)
faq.to_csv('../data/interim/faq-text-preprocessed.csv', index=False)

## < Hand-cleaning required at this point >

# Process Hand-cleaned Text Data

In [2]:
hand_cleaned = pd.read_csv('../data/interim/faq-hand-cleaned.csv')

## Separate Headings, Questions, Answers

In [3]:
# def find_heading(row):
#     entry = row['FAQ']
#     if entry.find('?') < 0:
#         row['heading'] = entry.strip()
#         row['FAQ'] = ''
#         return row
#     else:
#         q = entry.split('?')[0].split('. ')[-1]
#         heading = entry.split(q)[0]
#         row['heading'] = heading.strip()
#         row['FAQ'] = entry.replace(heading, '').strip()
#         return row

# # Pick out questions-and-answer pairs with a regular expression
# def qna_pairs(row):
#     relist = []
    
#     if row.heading is not '':
#         relist.append([row.Topic, row.heading.strip()])
    
#     pair = re.findall(r'([A-Z][^.]*\?\s)([^?]+(?:\.|\.\"|\.\)|\)\.|\.(?=[A-Z])|\)(?=I)|!))', row.FAQ)
#     if(pair):
#         for s in pair:
#             relist.append([s[0].strip(), s[1].strip()])
#     row['qna'] = relist
#     return row

# def process_text(df):
#     df = df.apply(find_heading, axis=1)
#     df = df.apply(qna_pairs, axis=1)
#     return df  

# faq = hand_cleaned.copy()
# faq.FAQ = faq.FAQ.apply(lambda x: x.replace('\n\n', ' '))
# faq = process_text(faq)
# faq

In [4]:
def pairs(row):
    topic_qna = pd.DataFrame()
    heading = True
    h = q = a = ''
    qna = []
    
    for item in row.FAQ:
        if heading:
            if '?' not in item:
                h += ' ' + item
            else:
                heading = False
                q = item
                a = ''
        elif '?' in item:
            qna.append([q.strip(), a.strip()])
            q = item
            a = ''
        else:
            a += ' ' + item
            
    if q:
        qna.append([q.strip(), a.strip()])

    if h:
        qna = [[row.Topic, h.strip()]] + qna
    row['heading'] = h.strip()
    row['qna'] = qna
    
    return row

new_faq = hand_cleaned.copy()
new_faq.FAQ = new_faq.FAQ.apply(lambda x: x.split('\n\n'))                
new_faq = new_faq.apply(pairs, axis=1)
new_faq

Topic  \
0    Affordable Homeownership                                           
1    Affordable Rental Opportunities                                    
2    Aircraft Noise                                                     
3    Airport                                                            
4    Alley Maintenance                                                  
5    Americans for the Arts Convention Scholarships                     
6    Americans with Disabilities Act (ADA)                              
7    Americans with Disabilities Act (ADA) Trails                       
8    Animal Noise (Barking Dogs)                                        
9    Animal Protection, Rules and Regulations                           
10   Animal Shelter                                                     
11   Ask-a-Librarian                                                    
12   Auction Licensing Instructions                                     
13   Backflow Prevention - Drinking Water                               
14   Bear Resistant Container Required                                  
15   Bears                                                              
16   Bicycling                                                          
17   Birth and Death Certificate Request                                
18   Block Party                                                        
19   Boulder Arts Commission Grants                                     
20   Boulder City Limits                                                
21   Boulder Civic Area                                                 
22   Boulder Convention and Visitors Bureau                             
23   Boulder Measures                                                   
24   Boulder Office of Emergency Management                             
25   Boulder Planning                                                   
26   Boulder Reservoir                                                  
27   Boulder Revised Code                                               
28   BoulderReads!                                                      
29   Broken Parking Meter or Pay Station                                
..                                   ...                                
220  Theater Rental and Marketing Assistance Grants                     
221  Towed/Impounded Vehicles                                           
222  Traffic Signals                                                    
223  Traffic Signs and Markings                                         
224  Train Noise and Quiet Zones                                        
225  Transit (Bus Service)                                              
226  Transportation Flood Recovery                                      
227  Transportation Projects                                            
228  Trash Concerns                                                     
229  Tree Branch Pick-Up                                                
230  Twenty Ninth Street                                                
231  Visitors                                                           
232  Volunteer Opportunities                                            
233  Volunteering at the Library                                        
234  Wage Issues                                                        
235  Waste Hauling Services                                             
236  Water Being Pumped or Diverted Off Private Property to Elsewhere   
237  Water Seepage                                                      
238  Water System Maintenance                                           
239  Water Utility Billing                                              
240  Water, Wastewater, Stormwater/Drainageway Flood Recovery           
241  Website Issues                                                     
242  Weed Control and Tall Vegetation                                   
243  Wildlife Issue or Sighting                  

## Re-Stack QnA Pairs

In [5]:
# stack = faq.apply(lambda x: pd.Series(x['qna']), axis=1).stack().reset_index(level=1, drop=True)
# stack.name = 'question'
# faq = faq.drop(['FAQ', 'qna', 'heading', 'Updated'], axis=1).join(stack).reset_index(drop=True)
# faq['answer'] = faq.question.apply(lambda x: x[1])
# faq['question'] = faq.question.apply(lambda x: x[0])
# faq

In [6]:
stack = new_faq.apply(lambda x: pd.Series(x['qna']), axis=1).stack().reset_index(level=1, drop=True)
stack.name = 'question'
new_faq = new_faq.drop(['FAQ', 'qna', 'heading', 'Updated'], axis=1).join(stack).reset_index(drop=True)
new_faq['answer'] = new_faq.question.apply(lambda x: x[1])
new_faq['question'] = new_faq.question.apply(lambda x: x[0])
new_faq

,Topic,Category,Department,question,answer
0,Affordable Homeownership,Housing,Housing,Affordable Homeownership,"The City of Boulder is committed to making housing in our community available to a variety of people, including those who could not otherwise purchase a home in Boulder's housing market. The Division of Housing offers opportunities for homeownership to those with low and moderate incomes - from down payment assistance on market rate homes to homes that must be sold at affordable prices."
1,Affordable Homeownership,Housing,Housing,How long does it take to become income-certified?,Generally up to two weeks.
2,Affordable Homeownership,Housing,Housing,Who needs to be on the application?,"Everyone who lives in the household or is part of the household, in addition to all people who will be listed on the property title. Household members should include all individuals: legal spouse, domestic partner, or common-law spouse; children (under 18 who reside with the applicant at least 50 percent of the time); and/or a significant other whom the applicant chooses to include as a member of his/her household, who will be occupying the house. The applicant's spouse must be included unless they are legally divorced or separated. Persons not counted include foster children, unborn children, and children under 18 who reside with the applicant less then 50 percent of the time."
3,Affordable Homeownership,Housing,Housing,How is annual income calculated?,"Annual income is defined as the anticipated total income for the next 12-month period received from all sources by each member (over the age of 18) of the household. It is assumed that today's circumstances will continue for the next 12 months. The applicant is required to verify this by either submitting a letter from his/her employer stating his/her annual wage or by completing an employer verification form. To obtain this form, call the Housing Division at 303-441-3157 ext. 2."
4,Affordable Homeownership,Housing,Housing,How is income verified for self-employed people?,Self-employed individuals should submit copies of their last three years of federal income-tax forms and must also submit an up-to-date profit/loss statement.
5,Affordable Homeownership,Housing,Housing,How are assets calculated?,"Assets are calculated based on annual income earned (interest), not the value of the asset itself."
6,Affordable Homeownership,Housing,Housing,Can I combine any programs?,"Neither First Home (20 percent down payment) nor the House to Home (15 percent down payment) Assistance Programs can be combined with a HomeWorks (permanently affordable) Home, since these homes have already been subsidized by the City of Boulder or the developer. The 3 percent solution may be used with HomeWorks."
7,Affordable Homeownership,Housing,Housing,Can I have a cosigner?,"Yes, a non-occupying cosigner will not be included in the household. However, if the cosigner's name appears on the title, her/his income must be included in the income certification application."
8,Affordable Homeownership,Housing,Housing,Do I have to be a first-time home buyer?,"To qualify for the down payment assistance programs, you must be a first-time home buyer, which means you have not owned a home in the last three years unless you have been divorced (in the last three years) or are a displaced homemaker. You do not have to be a first-time home buyer to purchase an Affordable Home."
9,Affordable Homeownership,Housing,Housing,Do I have to purchase in the city limits of Boulder?,Yes. Find out if the property is located in the Boulder City Limits.


In [7]:
faq = new_faq

In [8]:
faq.to_csv('../data/interim/faq-text-separated.csv', index=False)

# Write Excel files for QnA Maker, etc.

In [18]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('../data/interim/faq-qna-maker.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
faq[['question', 'answer']].to_excel(writer, sheet_name='faq-text-separated', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

# Write FAQ in 2 Columns, Q and A

In [9]:
faq[['question', 'answer']].to_csv('../data/interim/faq-two-columns.csv', index=False)